In [1]:
import src.view.pnl_view as pnlview
from multiprocessing import Pool, Process, Manager, cpu_count
from src.core.logger import Logger
from src.core.stopwatch import Stopwatch
from src.core.stopwatch_logger import StopwatchLogger
from src.synthetic_book_merger import SyntheticBookMerger
from src.synthetic_security_loader import SyntheticSecurityLoader
import src.core.domain.synthetic_security_fee
import src.sniper_opp_loader
from src.sniper_opp_loader import SniperOppLoader
from src.core.domain.security_loader import load_security

from src.view import data_view as dview
from src.view.opp_view import OppView, to_opp_view, display_opp_view
from src.view.cell_view import CellView
from src.view.color import Color, Colors, WebColors
from src.pcap_location_params import PCapLocationParams
from src.core.postgres_connection import PostgresConnection
from src.view.cell_view import CellView
from src.market_date import *

import pandas as pd
import numpy as np
import sqlite3
import time
import matplotlib.pyplot as plt
import os.path

# pandarallel.initialize(progress_bar=True, use_memory_fs=True)
pd.options.display.width = 920
pd.options.display.max_columns = 500
pd.options.display.max_rows = 220
pd.options.display.max_rows = 263

In [2]:
import importlib
import src.view.opp_view
import src.view.data_view as dview
import src.core.sort_utils as sort_utils
import src.core.numpy_utils as np_utils
import src.view.edge_summary_view
import src.view.opp_dur_count_view
import src.synthetic_security_loader
import src.synthetic_book_merger
import src.sniper_opp_loader
import src.sniper_opp_dur_summarizer
import src.view.opp_dur_view
importlib.reload(sort_utils)
importlib.reload(module=src.view.opp_view)
importlib.reload(module=dview)
importlib.reload(module=src.view.edge_summary_view)
importlib.reload(module=src.synthetic_security_loader)
importlib.reload(module=src.synthetic_book_merger)
importlib.reload(module=src.sniper_opp_loader)
importlib.reload(module=src.sniper_opp_dur_summarizer)
importlib.reload(module=src.view.opp_dur_view)
importlib.reload(module=src.view.opp_dur_count_view)
from src.view.opp_view import OppView
from src.view.edge_summary_view import get_edge_summary
from src.synthetic_security_loader import SyntheticSecurityLoader
from src.synthetic_book_merger import SyntheticBookMerger
from src.view.opp_dur_view import OppDurView

In [3]:
comex_apr_secs = load_security(360, date='04/01/2021')
comex_apr_secs.loc[comex_apr_secs['asset'] == 'GC'].sort_values('expiration')

,sid,symbol,expiration,sec_type,cme_sec_type,sec_group,asset,implied_matching,match_algo,display_factor,cash,tick_size,min_price,n_legs,segment_id
8341,43007,GCJ1-GSPJ106,2021-03-31 17:30:00,SPREAD,FUT,GL,GC,False,F,0.1,100.0,0.1,0.0,2,76
7393,35986,GCM1-GSPJ106,2021-03-31 17:30:00,SPREAD,FUT,GL,GC,False,F,0.1,100.0,0.1,0.0,2,76
998,54880,GCG2-GSPJ106,2021-03-31 17:30:00,SPREAD,FUT,GL,GC,False,F,0.1,100.0,0.1,0.0,2,76
5850,10280,GCZ1-GSPJ106,2021-03-31 17:30:00,SPREAD,FUT,GL,GC,False,F,0.1,100.0,0.1,0.0,2,76
5787,36097,GCQ1-GSPJ106,2021-03-31 17:30:00,SPREAD,FUT,GL,GC,False,F,0.1,100.0,0.1,0.0,2,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485,39231,GCZ5,2025-12-29 18:30:00,FUTURE,FUT,GC,GC,True,F,0.1,100.0,1.0,1.0,0,76
7856,24869,GCZ5-GCM6,2025-12-29 18:30:00,SPREAD,FUT,GC,GC,True,F,0.1,100.0,1.0,0.0,2,76
7791,19181,GCM6,2026-06-26 17:30:00,FUTURE,FUT,GC,GC,True,F,0.1,100.0,1.0,1.0,0,76
5750,73317,GCM6-GCZ6,2026-06-26 17:30:00,SPREAD,FUT,GC,GC,True,F,0.1,100.0,1.0,0.0,2,76


In [6]:
comex_apr_secs.loc[comex_apr_secs['symbol'].str.match('^SGU[A-Z][1-9]$')].sort_values('expiration')

,sid,symbol,expiration,sec_type,cme_sec_type,sec_group,asset,implied_matching,match_algo,display_factor,cash,tick_size,min_price,n_legs,segment_id
456,10086,SGCJ1,2021-04-28 06:15:00,FUTURE,FUT,AG,SGC,True,F,0.01,1000.0,5.0,5.0,0,76
1393,3986,SGCK1,2021-05-26 06:15:00,FUTURE,FUT,AG,SGC,True,F,0.01,1000.0,5.0,5.0,0,76
638,19117,SGCM1,2021-06-28 06:15:00,FUTURE,FUT,AG,SGC,True,F,0.01,1000.0,5.0,5.0,0,76
3470,2274,SGCQ1,2021-08-27 06:15:00,FUTURE,FUT,AG,SGC,True,F,0.01,1000.0,5.0,5.0,0,76
5764,10953,SGCV1,2021-10-27 06:15:00,FUTURE,FUT,AG,SGC,True,F,0.01,1000.0,5.0,5.0,0,76
3968,25423,SGCZ1,2021-12-29 06:15:00,FUTURE,FUT,AG,SGC,True,F,0.01,1000.0,5.0,5.0,0,76
836,3543,SGCG2,2022-02-24 06:15:00,FUTURE,FUT,AG,SGC,True,F,0.01,1000.0,5.0,5.0,0,76


In [12]:
from src.core.domain.security_loader import load_security
from src.pcap_location_params import PCapLocationParams

apr_secs = load_security(340, date='04/01/2021')
apr_secs

,sid,symbol,expiration,sec_type,cme_sec_type,sec_group,asset,implied_matching,match_algo,display_factor,cash,tick_size,min_price,n_legs,segment_id
0,108267,SASU1-SASK2,2021-08-13 18:20:00,SPREAD,FUT,AS,SAS,True,K,0.01,136.0,20.000,0.00,2,70
1,376965,UWFV1-UWFG2,2021-10-29 15:30:00,SPREAD,FUT,WP,UWF,True,F,0.01,0.0,25.000,0.00,2,70
2,773296,DFNK1-DFNM1,2021-05-27 15:30:00,SPREAD,FUT,UF,DFN,False,F,0.01,100.0,25.000,0.00,2,70
3,35681,KEH2-ZWN2,2022-03-14 17:01:00,SPREAD,FUT,KB,KE,True,K,1.00,50.0,0.250,0.00,2,70
4,28484,QC3U1,2021-08-27 17:01:00,FUTURE,FUT,05,QC3,False,K,1.00,0.0,0.250,0.25,0,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3881,273396,ZLF3-ZLQ3,2023-01-13 18:01:00,SPREAD,FUT,ZS,ZL,True,K,0.01,600.0,1.000,0.00,2,70
3882,644747,TRFM1-TRFH2,2021-06-30 18:20:00,SPREAD,FUT,LR,TRF,True,F,0.01,0.0,50.000,0.00,2,70
3883,207947,SASX1,2021-10-15 18:20:00,FUTURE,FUT,AS,SAS,True,K,0.01,136.0,20.000,20.00,0,70
3884,63770,XKK1-XKU1,2021-05-14 17:01:00,SPREAD,FUT,XK,XK,True,K,1.00,10.0,0.125,0.00,2,70


In [67]:
import datetime
from src.core.date_utils import daterange

secs = {}
pcap = PCapLocationParams(340)
for date in daterange(datetime.date(year=2021,month=4,day=1), stop=datetime.date(year=2021, month=5,day= 31), step=datetime.timedelta(days=1)):
    date_str = f'{date.year}-{date.month:02}-{date.day:02}'
    pcap_path = pcap.pcap_path(date=date_str)
    if pcap_path is None or not os.path.exists(pcap_path):
        continue
    sec_path = pcap.feather_file_path(name='security', date=date_str)
    print(f'Loading securities for {date}')
    secs[date] = pd.read_feather(sec_path)
# for date in {date: load_security(340, date=date) for date in ['']}

Loading securities for 2021-04-01
Loading securities for 2021-04-04
Loading securities for 2021-04-05
Loading securities for 2021-04-06
Loading securities for 2021-04-07
Loading securities for 2021-04-08
Loading securities for 2021-04-11
Loading securities for 2021-04-12
Loading securities for 2021-04-13
Loading securities for 2021-04-14
Loading securities for 2021-04-15
Loading securities for 2021-04-18
Loading securities for 2021-04-19
Loading securities for 2021-04-20
Loading securities for 2021-04-21
Loading securities for 2021-04-22
Loading securities for 2021-04-25
Loading securities for 2021-04-26
Loading securities for 2021-04-27
Loading securities for 2021-04-28
Loading securities for 2021-04-29
Loading securities for 2021-05-02
Loading securities for 2021-05-03
Loading securities for 2021-05-04
Loading securities for 2021-05-05
Loading securities for 2021-05-06
Loading securities for 2021-05-09
Loading securities for 2021-05-10
Loading securities for 2021-05-11
Loading securi

In [72]:
def get_sec_diff(date1, date2):
    return secs[date1].merge(secs[date2], how = 'outer' ,indicator=True).loc[lambda x : x['_merge']!='both']

def get_secs(date, regex = None):
    rv = secs[date]
    if regex is not None:
        rv = rv.loc[rv.symbol.str.match(regex)]
    return rv

sec_start_date = {}
dates = list(secs)
for i in range(0, len(dates) - 1):
    d1 = dates[i]
    d2 = dates[i +1]
    diff = get_sec_diff(d1, d2)
    ssd = diff.loc[diff['_merge'] == 'right_only'].copy()
    ssd.drop(columns='_merge', inplace=True)
    ssd['start_date'] = d1
    sec_start_date[d2] = ssd
# zs_secs = get_secs(datetime.date(year=2021, month=4,day=1), '.*ZS.*')

# for sec in zs_secs:
# zs_secs.sort_values('symbol')
# sec = get_secs(datetime.date(year=2021,month=5,day=2))
# sec.loc[sec['asset']=='ZS'].sort_values('expiration')
# sec_diff = get_sec_diff(datetime.date(year=2021,month=5,day=16), datetime.date(year=2021,month=5,day=12))
# sec_diff = sec_diff.loc[sec_diff['asset'] == 'ZS']
# sec_diff

In [80]:
all_secs = secs[dates[0]]
all_secs['start_date'] = datetime.date.min
all_secs = pd.concat(objs=[all_secs] + [sec_start_date[d] for d in sec_start_date])
# all_secs.to_sql('security_start_date', PostgresConnection(host='titan').connect())

In [86]:
pg_conn = PostgresConnection(host='titan').connect()
pd.read_sql('select * from sec_tmp', pg_conn).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          1 non-null      int64 
 1   id             1 non-null      int64 
 2   symbol         1 non-null      object
 3   channel        1 non-null      int64 
 4   exchange       1 non-null      object
 5   asset_class    1 non-null      object
 6   product_class  1 non-null      object
 7   first_date     1 non-null      object
 8   last_date      1 non-null      object
dtypes: int64(3), object(6)
memory usage: 200.0+ bytes


In [82]:
# all_secs.info()
all_secs[]
# all_secs.to_sql('security_start_date', PostgresConnection(host='titan').connect())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4365 entries, 0 to 3713
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   sid               4365 non-null   int64         
 1   symbol            4365 non-null   object        
 2   expiration        4365 non-null   datetime64[ns]
 3   sec_type          4365 non-null   category      
 4   cme_sec_type      4365 non-null   category      
 5   sec_group         4365 non-null   category      
 6   asset             4365 non-null   category      
 7   implied_matching  4365 non-null   object        
 8   match_algo        4365 non-null   category      
 9   display_factor    4365 non-null   float64       
 10  cash              4365 non-null   float64       
 11  tick_size         4365 non-null   float64       
 12  min_price         4365 non-null   float64       
 13  n_legs            4365 non-null   uint64        
 14  segment_id        4365 n

In [73]:
i = 0
for d in sec_start_date:
    display(sec_start_date[d])
    if i == 3:
        break
    i+=1

,sid,symbol,expiration,sec_type,cme_sec_type,sec_group,asset,implied_matching,match_algo,display_factor,cash,tick_size,min_price,n_legs,segment_id,start_date
3886,361304,ZW:BF H2-N2-Z2,2022-03-14 17:01:00,SPREAD,FUT,ZW,ZW,True,K,1.00,50.0,0.25,0.0,3,70,2021-04-01
3887,345843,ZW:BF K2-N2-Z2,2022-05-13 17:01:00,SPREAD,FUT,ZW,ZW,True,K,1.00,50.0,0.25,0.0,3,70,2021-04-01
3888,467594,SASH2-SASU2,2022-02-15 19:20:00,SPREAD,FUT,AS,SAS,True,K,0.01,136.0,20.00,0.0,2,70,2021-04-01
3889,353064,SASK2-SASU2,2022-04-14 18:20:00,SPREAD,FUT,AS,SAS,True,K,0.01,136.0,20.00,0.0,2,70,2021-04-01
3890,620111,SASU1-SASU2,2021-08-13 18:20:00,SPREAD,FUT,AS,SAS,True,K,0.01,136.0,20.00,0.0,2,70,2021-04-01
3891,19998,SASN2-SASU2,2022-06-15 18:20:00,SPREAD,FUT,AS,SAS,True,K,0.01,136.0,20.00,0.0,2,70,2021-04-01
3892,636666,SASU2,2022-08-15 18:20:00,FUTURE,FUT,AS,SAS,True,K,0.01,136.0,20.00,20.0,0,70,2021-04-01
3893,779798,SASN1-SASU2,2021-06-15 18:20:00,SPREAD,FUT,AS,SAS,True,K,0.01,136.0,20.00,0.0,2,70,2021-04-01
3894,204847,SASF2-SASU2,2021-12-15 19:20:00,SPREAD,FUT,AS,SAS,True,K,0.01,136.0,20.00,0.0,2,70,2021-04-01
3895,67329,SASQ2-SASU2,2022-07-15 18:20:00,SPREAD,FUT,AS,SAS,True,K,0.01,136.0,20.00,0.0,2,70,2021-04-01


,sid,symbol,expiration,sec_type,cme_sec_type,sec_group,asset,implied_matching,match_algo,display_factor,cash,tick_size,min_price,n_legs,segment_id,start_date


,sid,symbol,expiration,sec_type,cme_sec_type,sec_group,asset,implied_matching,match_algo,display_factor,cash,tick_size,min_price,n_legs,segment_id,start_date
3842,248112,DRTQ1,2021-08-17 21:00:00,FUTURE,FUT,LH,DRT,False,Q,0.01,100.0,1.0,1.0,0,70,2021-04-05


,sid,symbol,expiration,sec_type,cme_sec_type,sec_group,asset,implied_matching,match_algo,display_factor,cash,tick_size,min_price,n_legs,segment_id,start_date
3843,9448,DRSK1-DRSQ1,2021-05-19 18:40:00,SPREAD,FUT,RS,DRS,False,K,0.01,100.0,1.0,0.0,2,70,2021-04-06
3844,14520,DRSN1-DRSQ1,2021-07-21 18:40:00,SPREAD,FUT,RS,DRS,False,K,0.01,100.0,1.0,0.0,2,70,2021-04-06
3845,17746,DRSM1-DRSQ1,2021-06-16 18:40:00,SPREAD,FUT,RS,DRS,False,K,0.01,100.0,1.0,0.0,2,70,2021-04-06
3846,349772,DRSQ1,2021-08-18 18:40:00,FUTURE,FUT,RS,DRS,False,K,0.01,100.0,1.0,1.0,0,70,2021-04-06


In [10]:
apr_secs.loc[apr_secs['symbol'] == 'ZSH1']

,sid,symbol,expiration,sec_type,cme_sec_type,sec_group,asset,implied_matching,match_algo,display_factor,cash,tick_size,min_price,n_legs,segment_id


In [4]:
pg_conn = PostgresConnection(host='titan', username='postgres').connect()
secs = pd.read_sql('select * from security', pg_conn)
secs['']

,id,symbol,channel,exchange,asset_class,product_class,first_date,last_date


In [35]:
opp_w_bk_root=pd.read_feather('/home/john/spartan_store/pcaps/jupyter_cache/340/opp_w_bk_root_jun.feather').set_index(['market_date', 'symbol', 'opp_start_eid', 'synth_bk_eid']).sort_index()
opp_w_bk_root

t_time  id  bid_p_1  bid_q_1  bid_p_2  bid_q_2  bid_p_3  bid_q_3  ask_p_1  ask_q_1  ask_p_2  ask_q_2  ask_p_3  ask_q_3                    bk_dur                    fs_time                    ls_time                  bk_dur_lsn                  bk_dur_fsn                   opp_dur                 opp_dur_fsn                 opp_dur_lsn              opp_dur_thru            opp_dur_thru_fsn            opp_dur_thru_lsn  side  edge
market_date symbol                                             opp_start_eid synth_bk_eid                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
2021-06-01  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-Z... 91603         91603        2021-06-02 00:00:38.856343233   2     0.25        1     0.00        1    -0.25        2     2.50        2     2.75        1     3.00        1 0 days 00:00:00.000555842 2021-06-02 00:00:38.857062 2021-06-02 00:00:38.857151 -1 days +23:59:59.999748075 -1 days +23:59:59.999837075           0 days 00:00:00 -1 days +23:59:59.999281233 -1 days +23:59:59.999192233 0 days 00:00:00.000555842 -1 days +23:59:59.999837075 -1 days +23:59:59.999748075  Sell  0.25
                                                                             91607        2021-06-02 00:00:38.856899075   2     0.25        1    -0.25        3    -0.50        3     2.50        2     2.75        1     3.00        1 0 days 00:00:00.000040970 2021-06-02 00:00:38.857210 2021-06-02 00:00:38.857210 -1 days +23:59:59.999730045 -1 days +23:59:59.999730045 0 days 00:00:00.000555842 -1 days +23:59:59.999837075 -1 days +23:59:59.999748075 0 days 00:00:00.000596812 -1 days +23:59:59.999878045 -1 days +23:59:59.999789045  Sell  0.25
                                                                             91613        2021-06-02 00:00:38.856940045   2     0.00        1    -0.25        3    -0.50        2     2.50        2     2.75        1     3.00        1 0 days 00:00:00.000074254 2021-06-02 00:00:38.857320 2021-06-02 00:00:38.857320 -1 days +23:59:59.999694299 -1 days +23:59:59.999694299 0 days 00:00:00.000596812 -1 days +23:59:59.999878045 -1 days +23:59:59.999789045 0 days 00:00:00.000671066 -1 days +23:59:59.999952299 -1 days +23:59:59.999863299  None  0.00
                                                                             91617        2021-06-02 00:00:38.857014299   2     0.00        1    -0.25        3    -0.50        1     2.50        2     2.75        1     3.00        1 0 days 00:00:00.000001780 2021-06-02 00:00:38.857374 2021-06-02 00:00:38.857374 -1 days +23:59:59.999642079 -1 days +23:59:59.999642079 0 days 00:00:00.000671066 -1 days +23:59:59.999952299 -1 days +23:59:59.999863299 0 days 00:00:00.000672846 -1 days +23:59:59.999954079 -1 days +23:59:59.999865079  None  0.00
                                                                             91619        2021-06-02 00:00:38.857016079   2     0.00        1    -0.25        3    -0.50        1     2.25        1     2.50        1     2.75        2 0 days 00:00:00.000007748 2021-06-02 00:00:38.857453 2021-06-02 00:00:38.857453 -1 days +23:59:59.999570827 -1 days +23:59:59.999570827 0 days 00:00:00.000672846 -1 days +23:59:59.999954079 -1 days +23:59:59.999865079 0 days 00:00:00.000680594 -1 days +23:59:59.999961827 -1 days +23:59:59.999872827  None  0.00
...                                                                                                                 ...  ..      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...                       ...                        ...                     

In [7]:
# member_types = pd.DataFrame(data={
#     'id': [1,2,3,4,5],
#     'name': ['NON_MEMBER', ]
# })
# 'product_type', 'exchange', 'venue', 'security_type', 'embip_non_member', 'embip_member', 'rule_106j_non_member'
fee_data = [
    ['AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 'R106J_MEMBER', 0.23],
    ['AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 'R106J_NON_MEMBER', 0.51],
    ['AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 'IIP_MEMBER', 1.06],
    ['AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 'IIP_NON_MEMBER', 1.06],
    ['AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 'CBIP_MEMBER', 1.67],
    ['AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 'CBIP_NON_MEMBER', 1.67],
    ['AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 'NON_MEMBER', 1.95],

    ['AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 'R106J_MEMBER', 0.37],
    ['AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 'R106J_NON_MEMBER', 0.96],
    ['AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 'IIP_MEMBER', 2.06],
    ['AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 'IIP_NON_MEMBER', 2.06],
    ['AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 'CBIP_MEMBER', 3.84],
    ['AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 'CBIP_NON_MEMBER', 3.84],
    ['AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 'NON_MEMBER', 3.84],

    ['AGRICULTURAL', 'CME', 'GLBX', 'FUTURE', 'R106J_MEMBER', 0.51],
    ['AGRICULTURAL', 'CME', 'GLBX', 'FUTURE', 'R106J_NON_MEMBER', 0.51],
    ['AGRICULTURAL', 'CME', 'GLBX', 'FUTURE', 'IIP_MEMBER', 0.82],
    ['AGRICULTURAL', 'CME', 'GLBX', 'FUTURE', 'IIP_NON_MEMBER', 0.82],
    ['AGRICULTURAL', 'CME', 'GLBX', 'FUTURE', 'CBIP_MEMBER', 2.03],
    ['AGRICULTURAL', 'CME', 'GLBX', 'FUTURE', 'CBIP_NON_MEMBER', 2.03],
    ['AGRICULTURAL', 'CME', 'GLBX', 'FUTURE', 'NON_MEMBER', 2.03]
]

fee = pd.DataFrame(data=fee_data, columns=['product_type', 'exchange', 'venue', 'security_type', 'member_type', 'fee'])
fee
# fee = pd.DataFrame(data={
#     'product_type': ['agricultural'],
#     'exchange': ['cbot'],
#     'venue': ['globex'],
#     'security_type': ['FUT']
# })

,product_type,exchange,venue,security_type,member_type,fee
0,AGRICULTURAL,CBOT,GLBX,FUTURE,R106J_MEMBER,0.23
1,AGRICULTURAL,CBOT,GLBX,FUTURE,R106J_NON_MEMBER,0.51
2,AGRICULTURAL,CBOT,GLBX,FUTURE,IIP_MEMBER,1.06
3,AGRICULTURAL,CBOT,GLBX,FUTURE,IIP_NON_MEMBER,1.06
4,AGRICULTURAL,CBOT,GLBX,FUTURE,CBIP_MEMBER,1.67
5,AGRICULTURAL,CBOT,GLBX,FUTURE,CBIP_NON_MEMBER,1.67
6,AGRICULTURAL,CBOT,GLBX,FUTURE,NON_MEMBER,1.95
7,AGRICULTURAL,CBOT,GLBX,CALENDAR_SPREAD,R106J_MEMBER,0.37
8,AGRICULTURAL,CBOT,GLBX,CALENDAR_SPREAD,R106J_NON_MEMBER,0.96
9,AGRICULTURAL,CBOT,GLBX,CALENDAR_SPREAD,IIP_MEMBER,2.06


In [31]:
synthetic_security_leg = pd.DataFrame(data=[
    [1, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD',3],
    [2, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 4],
    [3, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 4],
    [4, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [4, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 2],
    [5, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 4],
    [6, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [6, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 1],
    [7, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [7, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 1],
    [8, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [8, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 1],
    [9, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 3],
    [10, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [10, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 1],
    [11, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [11, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 2],

    [12, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 4],
    [13, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 4],
    [14, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 3],

    [15, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [15, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 2],

    [16, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 4],
    [17, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 3],

    [18, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [18, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 2],

    [19, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 4],
    [20, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 4],
    [21, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 4],
    [22, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 4],

    [23, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 3],
    [24, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [24, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 2],
    [25, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 3],
    [26, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 4],
    [27, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 4],

    [28, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [28, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 2],
    [29, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [29, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 1],
    [30, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [30, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 2],

    [31, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [31, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 2],

    [32, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [32, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 1],

    [33, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [33, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 2],

    [34, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [34, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 2],

    [35, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [35, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 2],

    [36, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 3],

    [37, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [37, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 1],

    [38, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [38, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 1],

    [39, 'AGRICULTURAL', 'CBOT', 'GLBX', 'FUTURE', 2],
    [39, 'AGRICULTURAL', 'CBOT', 'GLBX', 'CALENDAR_SPREAD', 2],
], columns=['sid', 'product_type', 'exchange', 'venue', 'security_type', 'n_legs'])
synthetic_security_leg

,sid,product_type,exchange,venue,security_type,n_legs
0,1,AGRICULTURAL,CBOT,GLBX,CALENDAR_SPREAD,3
1,2,AGRICULTURAL,CBOT,GLBX,CALENDAR_SPREAD,4
2,3,AGRICULTURAL,CBOT,GLBX,CALENDAR_SPREAD,4
3,4,AGRICULTURAL,CBOT,GLBX,FUTURE,2
4,4,AGRICULTURAL,CBOT,GLBX,CALENDAR_SPREAD,2
5,5,AGRICULTURAL,CBOT,GLBX,CALENDAR_SPREAD,4
6,6,AGRICULTURAL,CBOT,GLBX,FUTURE,2
7,6,AGRICULTURAL,CBOT,GLBX,CALENDAR_SPREAD,1
8,7,AGRICULTURAL,CBOT,GLBX,FUTURE,2
9,7,AGRICULTURAL,CBOT,GLBX,CALENDAR_SPREAD,1


In [32]:
synthetic_security_leg.to_sql('synthetic_security_leg', PostgresConnection(host='titan').connect())

In [26]:
fee.to_sql('product_fee', PostgresConnection(host='titan').connect())

In [30]:
src.core.domain.synthetic_security_fee.load_synthetic_security_fees(synthetic_security_leg, fee.loc[fee['member_type'] == 'R106J_NON_MEMBER'])

,n_legs,fee,net_fee
sid,,,
1,3.0,0.96,2.88
2,4.0,0.96,3.84
3,4.0,0.96,3.84
4,4.0,1.47,2.94
5,4.0,0.96,3.84
6,3.0,1.47,1.98
7,3.0,1.47,1.98
8,3.0,1.47,1.98
9,3.0,0.96,2.88


In [29]:
synth_fee = pd.merge(synthetic_security_leg, fee, on=['product_type', 'exchange', 'venue', 'security_type'], how='left')
synth_fee = synth_fee.loc[synth_fee['member_type'] == 'R106J_NON_MEMBER']
synth_fee['net_fee'] = synth_fee['n_legs'] * synth_fee['fee']
# synth_fee.loc[synth_fee['sid'] == 1][['net_fee']]
synth_fee.groupby('sid').sum()
# synth_fee.loc[synth_fee['member_type'] == 'R106J_MEMBER'].groupby('sid').apply(lambda x: x['n_legs'] * 

,n_legs,fee,net_fee
sid,,,
1,3.0,0.96,2.88
2,4.0,0.96,3.84
3,4.0,0.96,3.84
4,4.0,1.47,2.94
5,4.0,0.96,3.84
6,3.0,1.47,1.98
7,3.0,1.47,1.98
8,3.0,1.47,1.98
9,3.0,0.96,2.88


In [3]:
pd.read_feather('/home/john/spartan_store/pcaps/340/2021-06-09_340_24h_feather/security.feather')

,sid,symbol,expiration,sec_type,cme_sec_type,sec_group,asset,implied_matching,match_algo,display_factor,cash,tick_size,min_price,n_legs,segment_id
0,95209,EHG2-EHN3,2022-02-03 18:01:00,SPREAD,FUT,EH,EH,True,K,0.001,29000.0,1.000,0.000,2,70
1,634962,UNOM1,2021-06-24 15:30:00,FUTURE,FUT,UF,UNO,False,F,0.010,100.0,25.000,25.000,0,70
2,34696,ZSN2-ZSH3,2022-07-14 17:01:00,SPREAD,FUT,ZS,ZS,True,K,1.000,50.0,0.250,0.000,2,70
3,7456,EH:FS 04M U1,2021-09-03 17:01:00,SPREAD,FUT,EH,EH,False,K,0.001,29000.0,1.000,0.000,4,70
4,797051,MFCN1-MFCQ1,2021-07-29 15:30:00,SPREAD,FUT,UF,MFC,False,F,0.010,100.0,25.000,0.000,2,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,20301,ZWN1,2021-07-14 17:01:00,FUTURE,FUT,ZW,ZW,True,K,1.000,50.0,0.250,0.250,0,70
3616,245584,SOM:SI Q1-Q1-Q1,2021-08-13 17:01:00,SPREAD,FUT,ZS,SOM,True,K,0.010,500.0,0.250,0.000,3,70
3617,693240,XKF3,2023-01-13 18:01:00,FUTURE,FUT,XK,XK,True,K,1.000,10.0,0.125,0.125,0,70
3618,37017,XKQ2-XKX4,2022-08-12 17:01:00,SPREAD,FUT,XK,XK,True,K,1.000,10.0,0.125,0.000,2,70


In [2]:
df = SU.load_sniper_opps('zs')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 21422 entries, ('0', Timestamp('2020-08-23 00:00:00')) to ('99', Timestamp('2021-06-24 00:00:00'))
Data columns (total 48 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   market_date      21422 non-null  datetime64[ns] 
 1   run_id           21422 non-null  int64          
 2   opp_id           21422 non-null  object         
 3   eid              21422 non-null  int64          
 4   t_time           21422 non-null  datetime64[ns] 
 5   side             21276 non-null  object         
 6   poly             21422 non-null  object         
 7   has_futures      21422 non-null  bool           
 8   min_days         21422 non-null  int64          
 9   max_days         21422 non-null  int64          
 10  min_fut_vol      21422 non-null  int64          
 11  entry_ticks      21422 non-null  int64          
 12  entry_pnl        21276 non-null  float64        
 13  ent

In [20]:
len(df.loc[df['is_direct'] == True])

21276

In [18]:
fdf = df.loc[df['is_direct'] == False].drop(columns=['t_time', 'has_futures', 'min_days', 'max_days'])
fdf = fdf.loc[fdf['entry_ticks'] > 2]
fdf['entry_qty_gte5'] = fdf['entry_qty'] >= 1
qty_grps = fdf[['entry_qty_gte5', 'entry_qty', 'entry_ticks', 'lsn_win_us']].groupby('entry_qty_gte5')
quantiles = [.1, .2, .3, .5, .7, .8, .9]
SU.get_opp_duration_quantiles(qty_grps, quantiles, ['lsn_win_us'])

,n_opps,0.10_lsn_win_us,0.20_lsn_win_us,0.30_lsn_win_us,0.50_lsn_win_us,0.70_lsn_win_us,0.80_lsn_win_us,0.90_lsn_win_us


In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import sqlite3
import SniperUtils as SU
from src.core import IterUtils
from src.view import DataView as dview

In [2]:
sql_conn = sqlite3.connect('/md/SIM_DBs/sim_zs.sqlite')
root_df = pd.read_sql('select * from sniper_opps', sql_conn)
df = SU.cast_sniper_opps_cols(root_df)
df['tot_ticks'] = df['entry_ticks'] * df['entry_qty']
df = SU.add_constrs(df)
df = SU.remove_double_dips(df)
df['lsn_win_us'] = df['lsn_win'].apply(lambda x: x.microseconds)

In [4]:
fdf = df.loc[df['entry_ticks'] > 2]
qty_grps = fdf[['entry_qty', 'entry_ticks', 'lsn_win_us']].groupby('entry_qty')
quantiles = [.1, .2, .3, .5, .7, .8, .9]
def get_quantiles(g):
    row = []
    for q in quantiles:
        gl = g.loc[g['entry_ticks'] > 100]
        row.append(gl['lsn_win'].quantile(q))
    return row

qt1=qty_grps.quantile(.1)
qt1.style.format({'lsn_win_us': lambda x: f'{x:.2f}'})
#     return [g['lsn_win'].quantile(q) for q in quantiles]
# l = qty_grps.apply(get_quantiles)
# pd.DataFrame(data=[l], columns=[str(q) for q in quantiles])
# pd.DataFrame(data=qty_grps.apply(get_quantiles), columns=[str(q) for q in quantiles])
# pd.DataFrame(data=qty_grps.apply(get_quantiles), columns=[str(q) for q in quantiles])
# for grp in qty_grps.groups:
#     qty_grps.get_group(grp)
#     break

,entry_ticks,lsn_win_us
entry_qty,,
1,3.000000,68.00
2,3.000000,45.60
3,3.000000,50.00
4,3.000000,39.70
5,3.000000,59.90
6,3.000000,38.40
7,3.000000,757.40
8,3.000000,252.80
9,3.400000,345.40


In [5]:
qts = [qty_grps.quantile(q) for q in quantiles]
summ_data = {}
qt_idx = 0
for q in quantiles:
    summ_data[f'{q:.2f}'] = qts[qt_idx]['lsn_win_us']
    qt_idx += 1
pd.DataFrame(data=summ_data)
# summ = pd.DataFrame(data={'0.1': qts[0]['lsn_win_us']})
# summ

,0.10,0.20,0.30,0.50,0.70,0.80,0.90
entry_qty,,,,,,,
1,68.0,1399.2,128777.8,981489.0,993829.8,997038.6,999244.2
2,45.6,79.2,330.2,22255.0,993324.3,997336.0,999464.5
3,50.0,104.0,884.0,977676.0,997079.0,999204.0,999636.0
4,39.7,106.6,169.1,3818.5,988882.4,997533.0,998955.1
5,59.9,86.4,245.1,3395.5,999005.5,999457.0,999590.7
6,38.4,70.0,161.0,1213.0,6729.4,995805.8,998161.2
7,757.4,1128.6,1567.8,999181.0,999691.6,999788.8,999845.6
8,252.8,479.6,1200.2,3629.0,799730.6,999004.4,999501.2
9,345.4,626.8,908.2,1471.0,2033.8,2315.2,2596.6
